In [1]:
from preprocesamiento import training_seq, SEQ_LEN
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

#es el tamaño del diccionario
OUTPUT_UNITS = 38
NUM_UNITS = 256
LOSS =  nn.CrossEntropyLoss(reduction = 'none')
LR = 0.01
EPOCH = 10
BATCH_SIZE = 32
SAVE_MODEL_PATH = 'model.pt'

#clase para generar una red lstm
class model_lstm(nn.Module):
    def __init__(self, output_units, num_units):
        super(model_lstm, self).__init__()
        self.lstm = nn.LSTM(output_units, num_units)
        self.dropout = nn.Dropout(0.1)
        self.linear = nn.Linear(num_units, output_units)
        #agragamos soft max
        self.softmax = nn.Softmax(dim = 1)

    def feed_forward(self, x):
        output, _ = self.lstm(x)
        output = self.dropout(output)
        output = self.linear(output)
        #agregamos soft max
        output = self.softmax(output)

        return output

def train(output_units = OUTPUT_UNITS, num_units = NUM_UNITS, loss = LOSS, lr = LR):
    inputs, targets = training_seq(SEQ_LEN)
    targets = np.eye(38)[targets]

    #los pasamos a tensores
    inputs = torch.from_numpy(inputs).float()
    targets = torch.from_numpy(targets).long()

    #creamos la red y el optimizador
    red_lstm = model_lstm(output_units, num_units)
    optimizer = optim.Adam(red_lstm.parameters(), lr = lr)

    #entrenamos
    for epoch in range(EPOCH):
        loss_total = 0

        #mini batch
        for i in range(0, len(inputs), BATCH_SIZE):           
            batch_inputs = inputs[i:i+BATCH_SIZE]
            batch_targets = targets[i:i+BATCH_SIZE]
            
            print(f'batch_inputs = {batch_inputs.shape}')
            print(f'batch_targets = {batch_targets.shape}')

            optimizer.zero_grad()

            #hacemos feed forward
            outputs = red_lstm.feed_forward(batch_inputs)           
            print(f'outputs: {outputs.shape}')

            #tomamos la salida con mayor probabilidad
            # outputs = torch.argmax(outputs, dim = 1)
            print(f'outputs: {outputs.shape}')
            # loss_val = loss(outputs.permute(0, 1, 2), batch_targets)
            loss_val = loss(outputs, batch_targets)

            #backpropagation
            loss_val.sum().backward()
            optimizer.step()

            loss_total += loss_val.sum().item()

        print(f'Epoch: {epoch + 1} \t Loss: {loss_total:.5f}')

    #guardamos el modelo
    torch.save(red_lstm.state_dict(), SAVE_MODEL_PATH)
    #guardamos el modelo
    torch.save(red_lstm.state_dict(), SAVE_MODEL_PATH)

In [2]:
train()

batch_inputs = torch.Size([32, 32, 38])
batch_targets = torch.Size([32, 38])
outputs: torch.Size([32, 32, 38])
outputs: torch.Size([32, 32, 38])
batch_inputs = torch.Size([32, 32, 38])
batch_targets = torch.Size([32, 38])
outputs: torch.Size([32, 32, 38])
outputs: torch.Size([32, 32, 38])
batch_inputs = torch.Size([32, 32, 38])
batch_targets = torch.Size([32, 38])
outputs: torch.Size([32, 32, 38])
outputs: torch.Size([32, 32, 38])
batch_inputs = torch.Size([32, 32, 38])
batch_targets = torch.Size([32, 38])
outputs: torch.Size([32, 32, 38])
outputs: torch.Size([32, 32, 38])
batch_inputs = torch.Size([32, 32, 38])
batch_targets = torch.Size([32, 38])
outputs: torch.Size([32, 32, 38])
outputs: torch.Size([32, 32, 38])
batch_inputs = torch.Size([32, 32, 38])
batch_targets = torch.Size([32, 38])
outputs: torch.Size([32, 32, 38])
outputs: torch.Size([32, 32, 38])
batch_inputs = torch.Size([32, 32, 38])
batch_targets = torch.Size([32, 38])
outputs: torch.Size([32, 32, 38])
outputs: torch.Size

KeyboardInterrupt: 